In [1]:
import torch
import torch.optim
import os
import model
import numpy as np
import cv2 as cv

scale_factor = 12

DCE_net = model.enhance_net_nopool(scale_factor)

DCE_net.load_state_dict(torch.load('checkpoints/attention_reduced_rr/Epoch9.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [2]:
def enhance_image(frame):
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    scale_factor = 12
    data_lowlight = frame

    data_lowlight = (np.asarray(data_lowlight)/255.0)

    data_lowlight = torch.from_numpy(data_lowlight).float()

    h=(data_lowlight.shape[0]//scale_factor)*scale_factor
    w=(data_lowlight.shape[1]//scale_factor)*scale_factor
    data_lowlight = data_lowlight[0:h,0:w,:]
    data_lowlight = data_lowlight.permute(2,0,1)
    
    data_lowlight = data_lowlight.unsqueeze(0)
    enhanced_image,params_maps = DCE_net(data_lowlight)

    params_maps = params_maps.detach().numpy()[0]
    params_maps = np.einsum('kij->ijk',params_maps)
    params_maps = ( params_maps + 1 )* 0.5
    enhanced_image = enhanced_image.detach().numpy()[0]
    enhanced_image = np.einsum('kij->ijk',enhanced_image)
  
    return enhanced_image, params_maps

cap = cv.VideoCapture(0)
ret, frame = cap.read()
cap.release()

In [3]:


fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 30.0, (640, 480))

cap = cv.VideoCapture(0)
display_light_curve_map = False

while True:
    ret, frame = cap.read()
    
    frame_orig = frame.copy()
    enhanced_frame, light_curve_map = enhance_image(frame)    
    frame_orig = frame_orig[:,:-4] / 255
    
    if display_light_curve_map == False:
        v_img = np.concatenate([frame_orig, enhanced_frame], axis=1)
    else:
        v_img = np.concatenate([frame_orig, light_curve_map, enhanced_frame], axis=1)
    cv.imshow("enhanced", v_img)

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    if cv.waitKey(1) & 0xFF == ord("l"):
        display_light_curve_map = not display_light_curve_map 

cap.release()
cv.destroyAllWindows()

out.release()